In [6]:

from langchain_community.document_loaders import TextLoader,PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader
loader= PyPDFLoader("research.pdf")
documents=loader.load()
documents[0].page_content

'Proceedings of 15th IOE Graduate Conference\nPeer Reviewed\nYear: 2024 Month: May Volume: 15\nISSN: 2350-8914 (Online), 2350-8906 (Print)\nNepali Sign Language Letter Detection and Finger Spelling Using\nMediapipe and CNN\nGuptaraj Shrestha a, Nishan Thing b, Prajita Dhakal c, Prasanga Dahal d, Pukar Karki e, Manoj\nKumar Guragai f\na,b,c,d,e,f Department of Electronics and Computer Engineering, Purwanchal Campus, IOE, TU, Nepal\n a 076bct034@ioepc.edu.np , b 076bct052@ioepc.edu.np , c 076bct057@ioepc.edu.np , d 076bct058@ioepc.edu.np , e pukar@ioepc.edu.np , f\nmanojkguragai@ioepc.edu.np\nAbstract\nThis paper describes the development of a machine learning model to translate Nepali Sign Language (NSL) gestures into\ncorresponding Nepali text format. The system utilizes computer vision and deep learning techniques, specifically a Convolutional\nNeural Network (CNN) model trained on a dataset of over 2,500 images per label with total of 50 labels. The training accuracy was\n99%, wherea

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
docs =text_splitter.split_documents(documents)


In [12]:
docs

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-08-22T12:54:44+05:45', 'author': 'Guptaraj Shrestha, Nishan Thing, Prajita Dhakal, Prasanga Dahal, Pukar Karki, Manoj Kumar Guragain', 'title': 'Nepali Sign Language Letter Detection and Finger Spelling Using Mediapipe and CNN', 'subject': '14th IOE Graduate Conference', 'keywords': 'Nepali Sign Language, Machine Learning, Convolutional Neural Network, MediaPipe', 'moddate': '2024-08-22T12:54:44+05:45', 'trapped': '/False', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'source': 'research.pdf', 'total_pages': 5, 'page': 0, 'page_label': '166'}, page_content='Proceedings of 15th IOE Graduate Conference\nPeer Reviewed\nYear: 2024 Month: May Volume: 15\nISSN: 2350-8914 (Online), 2350-8906 (Print)\nNepali Sign Language Letter Detection and Finger Spelling Using\nMediapipe and CNN\nGuptaraj Shrestha a, Nishan Thing b, Prajit

In [16]:
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import FAISS

llm= OllamaEmbeddings(model="mistral")

db=FAISS.from_documents(docs,llm)
db

In [17]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
    Give the correct answer to the question based on the context provided.
    think step by step and provide the final answer.
    if answer is correct i will give 5 start rating to you.
    <context>
    {context}
    </context>
    Question:{input}
    """
    
)

In [34]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.llms import Ollama
model = Ollama(model="mistral")
document_chain = create_stuff_documents_chain(llm=model, prompt=prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\n    Give the correct answer to the question based on the context provided.\n    think step by step and provide the final answer.\n    if answer is correct i will give 5 start rating to you.\n    <context>\n    {context}\n    </context>\n    Question:{input}\n    '), additional_kwargs={})])
| Ollama(model='mistral')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [36]:
retriever_db = db.as_retriever(search_kwargs={"k": 1})
retriever_db

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000018B6FAFC2F0>, search_kwargs={'k': 1})

In [38]:
from langchain.chains import create_retrieval_chain
retrieval_chain =create_retrieval_chain(
    retriever_db,document_chain)

In [40]:
response =retrieval_chain.invoke({"input": "What is the main focus of the research paper?"})
response['answer']

' The main focus of the research paper is on developing a CNN model to identify Nepali Sign Language (NSL) signs from video frames of gestures. This is inferred from the context provided, as it mentions that the training process is focused on this task and the dataset used for this purpose is a Hand Landmark Image of the Nepali alphabet.'